In [21]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("")
sys.path.append("../..")

In [30]:
import statsmodels as sm

In [24]:
2 + 0.15*np.arange(1, 21, 1) + np.random.randn(20)

array([3.04597623, 2.26982297, 1.92137723, 3.83726139, 2.93166713,
       3.95894068, 1.44447605, 2.60294754, 1.12430882, 4.24618511,
       4.72027933, 5.35836859, 4.64784992, 3.66981756, 4.54897754,
       4.60737062, 4.59307652, 4.88819735, 6.08950047, 5.8778822 ])

In [29]:
#In this example let us produce forecasts for 100 randomly generated timeseries
fh = 6 #The forecasting horizon examined
frq = 1 #The frequency of the data
data_train = np.zeros((100, 20-fh))
data_test = np.zeros((100, fh))
print(data_train.shape, data_test.shape)
for i in range(100):
  data_all = 2 + 0.15*np.arange(1, 21, 1) + np.random.randn(20) 
  data_train[i,:] = data_all[:-fh]
  data_test[i,:] = data_all[-fh:]
data_train.shape, data_test.shape

(100, 14) (100, 6)


((100, 14), (100, 6))

In [ ]:
def SMAPE(actual: np.array, predicted: np.array) -> float:
    """ Calculating Symmetric Mean Absolute Prediction Error """
    nz = np.where(actual > 0)
    Pz = predicted[nz]
    Az = actual[nz]
    return np.mean(2.0 * np.abs(Az - Pz) / (np.abs(Az) + np.abs(Pz)))

In [ ]:
def smape_cal(outsample, forecasts): # should calculate for abunch of ts at once
    return (np.abs(outsample - forecast)*200)/(np.abs(outsample) + np.abs(forecasts))

In [ ]:
def mase_cal(insample, outsample, forecasts, frequency):
    masep = np.mean(np.abs(np.diff(insample, n=frequency), axis=1))
    mase = np.abs(outsample-forecasts) / masep
    return mase

In [ ]:
def naive_seasonal(input, fh, frequency):
    return 0

In [ ]:
def seasonality_test(ts, ppy):
    t_crit = 1.645
    if len(ts) < 3*ppy:
        test_seasonal = False
    else:
        xacf = sm.tsa.stattols.acf(ts)[1:]

        clim = t_crit / np.sqrt(len(x)) * sqrt(np.cumsum([1, 2*xacf**2]))
        if np.abs(xacf[ppy]) > clim[ppy]
            test_seasonal = True
    return test_seasonal

In [ ]:
def benchmarks(ts, fh, frequency):
    ppy = frequency
    st = False
    if ppy > 1:
        st = seasonality_test(ts, ppy)
    if st:
        dec = sm.tsa.seasonal_decompose(ts, model=multiplicative)
        des_ts = ts / dec.seasonal
        SIout = dec.seasonal[(-ppy+1):]
    else:
        des_ts = ts
        SIout = np.ones(fh)

    naive = np.repeat(ts[-1], fh)
    naive2 = np.repeat(des_ts[-1], fh)*SIout

    return naive, naive2

In [ ]:
names = ["naive", "naive2"]
scores = np.zeros((3, len(names)))

for i in range(data_train.shape[0]):
    forecasts = benchmarks(data_train[i], fh=6, frequency=1)
    for i, f in enumerate(forecasts)
    scores[i,]



In [ ]:

Names_benchmarks <- c("Naive", "sNaive", "Naive2", "SES", "Holt", "Damped", "Theta", "Com")
Total_smape=Total_mase <- array(NA,dim = c(length(Names_benchmarks), fh, length(data_train)))
#Methods, Horizon, time-series
for (i in 1:length(data_train)){
  
  insample <- data_train[[i]]
  outsample <- data_test[[i]]
  forecasts <- Benchmarks(input=insample, fh=fh)
  
  #sMAPE
  for (j in 1:length(Names_benchmarks)){
    Total_smape[j,,i] <- smape_cal(outsample, forecasts[[j]]) #j the # of the benchmark
  }
  #MASE
  for (j in 1:length(Names_benchmarks)){
    Total_mase[j,,i] <- mase_cal(insample, outsample, forecasts[[j]]) #j the # of the benchmark
  }
  
}

print("########### sMAPE ###############")
for (i in 1:length(Names_benchmarks)){
  print(paste(Names_benchmarks[i], round(mean(Total_smape[i,,]), 3)))
}
print("########### MASE ################")
for (i in 1:length(Names_benchmarks)){
  print(paste(Names_benchmarks[i], round(mean(Total_mase[i,,]), 3)))
}
print("########### OWA ################")
for (i in 1:length(Names_benchmarks)){
  print(paste(Names_benchmarks[i],
              round(((mean(Total_mase[i,,])/mean(Total_mase[3,,]))+(mean(Total_smape[i,,])/mean(Total_smape[3,,])))/2, 3)))
}

